# Introduction to ITK Segmentation in SimpleITK Notebooks

<b>Goal</b>: To become familiar with basic segmentation algorithms available in ITK, and interactively explore their parameter space.

Image segmentation filters process an image to partition it into (hopefully) meaningful regions. The output is commonly an image of integers where each integer can represent an object. The value 0 is commonly used for the background, and 1 (sometimes 255) for a foreground object.


In [ ]:
library(SimpleITK)

source("downloaddata.R")

We start by loading our data and looking at it.

In [ ]:
img_T1 <- ReadImage(fetch_data("nac-hncma-atlas2013-Slicer4Version/Data/A1_grayT1.nrrd"))

# To visualize the labels image in color (RGB) we need to rescale the intensities to [0-255]
img_T1_255 <- Cast(RescaleIntensity(img_T1), "sitkUInt8")

Show(img_T1)

## Thresholding

Thresholding is the most basic form of segmentation. It simply labels the pixels of an image based on the intensity range without considering geometry or connectivity.

In [ ]:
seg <- img_T1>200

#Overlay segmentation onto the original image using the default alpha blending value of 0.5.
Show(LabelOverlay(img_T1_255, seg))

In [ ]:
# A slightly more targeted thresholding approach with the values of interest inside a range
seg <- BinaryThreshold(img_T1, lowerThreshold=100, upperThreshold=400, 
                       insideValue=1, outsideValue=0)

Show(LabelOverlay(img_T1_255, seg))

### Selecting thresholds
SimpleITK has a number of histogram based methods for automatic threshold selection for a bimodal distribution. These include <a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1HuangThresholdImageFilter.html">Huang</a>, <a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1MaximumEntropyThresholdImageFilter.html">MaximumEntropy</a>, <a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1TriangleThresholdImageFilter.html">Triangle</a>, and the popular <a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1OtsuThresholdImageFilter.html">Otsu's</a> method. These methods create a histogram then use a heuristic to determine the threshold value which separates the foreground from background.

In [ ]:
otsu_filter <- OtsuThresholdImageFilter()
otsu_filter$SetInsideValue(0)
otsu_filter$SetOutsideValue(1)
seg <- otsu_filter$Execute(img_T1)

cat(otsu_filter$GetThreshold())

Show(LabelOverlay(img_T1_255, seg))

## Region Growing Segmentation

The first step of improvement upon the naive thresholding is a class of algorithms called region growing. These include:
<ul>
  <li><a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1ConnectedThresholdImageFilter.html">ConnectedThreshold</a></li>
  <li><a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1ConfidenceConnectedImageFilter.html">ConfidenceConnected</a></li>
  <li><a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1VectorConfidenceConnectedImageFilter.html">VectorConfidenceConnected</a></li>
  <li><a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1NeighborhoodConnectedImageFilter.html">NeighborhoodConnected</a></li>
</ul>

We used an external program, 3D Slicer, to determine that index (132,142,96) was a good seed for the left lateral ventricle.

In [ ]:
seed <- c(132,142,96)

# visualize the seed with the original image: create a "segmentation" image with the seed 
# dilated so that it is clearly visible when overlaid onto the original image
seg <- Image(img_T1$GetSize(), "sitkUInt8")
seg$CopyInformation(img_T1)
seg$SetPixel(seed,1)
seg <- BinaryDilate(seg, 3)

Show(LabelOverlay(img_T1_255, seg))

Now segment using the connected threshold functionality, pixels that are connected to the seed(s) and lie within a given range of values.

In [ ]:
seg <- ConnectedThreshold(img_T1, seedList=list(seed), lower=100, upper=190)

Show(LabelOverlay(img_T1_255, seg))

Improving upon this is the ConfidenceConnected filter, which uses the initial seed(s) or current segmentation to estimate the thresholds based on the intensity mean and standard deviation: $\mu\pm c\sigma$. 

The constant $c$ from the formula above is the "multiplier" function parameter.

In [ ]:
seg <- ConfidenceConnected(img_T1, seedList=list(seed),
                           numberOfIterations=1,
                           multiplier=2.5,
                           initialNeighborhoodRadius=1,
                           replaceValue=1)

Show(LabelOverlay(img_T1_255, seg))

If we have a multi-channel image we can possibly improve on the results above by using more channels. In our case we can use both T1 and T2 images to perform the segmentation. This is a multi-dimensional version of the previous filter with the "multiplier" used in conjunction with the Mahalanobis distance.

In [ ]:
img_T2 <- ReadImage(fetch_data("nac-hncma-atlas2013-Slicer4Version/Data/A1_grayT2.nrrd"))

img_multi <- Compose(img_T1, img_T2)
seg <- VectorConfidenceConnected(img_multi, seedList=list(seed),
                                 numberOfIterations=1,
                                 multiplier=2.5,
                                 initialNeighborhoodRadius=1)

Show(LabelOverlay(img_T1_255, seg))

## Fast Marching Segmentation

The FastMarchingImageFilter implements a fast marching solution to a simple level set evolution problem (eikonal equation). In this example, the speed term used in the differential equation is provided in the form of an image. The speed image is based on the gradient magnitude and mapped with the bounded reciprocal $1/(1+x)$.

In [ ]:
seed <- c(132,142,96)
feature_img <- GradientMagnitudeRecursiveGaussian(img_T1, sigma=.5)
speed_img <- BoundedReciprocal(feature_img) # compute 1/(1+x) for each pixel

Show(speed_img)

The output of the FastMarchingImageFilter is a <b>time-crossing map</b> that indicates, for each pixel, how much time it would take for the front to arrive at the pixel location.

In [ ]:
fm_filter <- FastMarchingBaseImageFilter()
fm_filter$SetTrialPoints(list(seed))
fm_filter$SetStoppingValue(1000)
fm_img <- fm_filter$Execute(speed_img)

Show(Threshold(fm_img,
               lower=0.0,
               upper=fm_filter$GetStoppingValue(),
               outsideValue=fm_filter$GetStoppingValue()+1))

In the next cell you need to select the arrival time to obtain the desired segmentation (we start at 500 which is an over-segmentation, zero is the lower bound - rerun this cell to search for the desired threshold - binary search comes to mind)  

In [ ]:
arrival_time <- 500
seg <- fm_img<arrival_time

Show(LabelOverlay(img_T1_255, seg))

## Level-Set Segmentation

There are a variety of level-set based segmentation filters available in SimpleITK:
<ul>
<li><a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1GeodesicActiveContourLevelSetImageFilter.html">GeodesicActiveContour</a></li>
<li><a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1ShapeDetectionLevelSetImageFilter.html">ShapeDetection</a></li>
<li><a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1ThresholdSegmentationLevelSetImageFilter.html">ThresholdSegmentation</a></li>
<li><a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1LaplacianSegmentationLevelSetImageFilter.html">LaplacianSegmentation</a></li>
<li><a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1ScalarChanAndVeseDenseLevelSetImageFilter.html">ScalarChanAndVese</a></li>
</ul>

First we create a label image from our seed.

In [ ]:
seed <- c(132,142,96)

seg <- Image(img_T1$GetSize(), "sitkUInt8")
seg$CopyInformation(img_T1)
seg$SetPixel(seed, 1)
seg <- BinaryDilate(seg, 3)

Use the seed to estimate a reasonable threshold range.

In [ ]:
stats <- LabelStatisticsImageFilter()
# ITK and therefore SimpleITK return an empty image for this execute - we'll just ignore it
dev_null <- stats$Execute(img_T1, seg)

factor <- 3.5
lower_threshold <- stats$GetMean(label=1)-factor*stats$GetSigma(label=1)
upper_threshold <- stats$GetMean(label=1)+factor*stats$GetSigma(label=1)
cat(sprintf("lower threshold: %f\nupper threshold %f\n",lower_threshold,upper_threshold))

In [ ]:
init_ls <- SignedMaurerDistanceMap(seg, TRUE, TRUE)

In [ ]:
lsFilter <- ThresholdSegmentationLevelSetImageFilter()
lsFilter$SetLowerThreshold(lower_threshold)
lsFilter$SetUpperThreshold(upper_threshold)
lsFilter$SetMaximumRMSError(0.02)
lsFilter$SetNumberOfIterations(1000)
lsFilter$SetCurvatureScaling(.5)
lsFilter$SetPropagationScaling(1)
lsFilter$ReverseExpansionDirectionOn()
ls <- lsFilter$Execute(init_ls, Cast(img_T1, "sitkFloat32"))
print(lsFilter)

In [ ]:
Show(LabelOverlay(img_T1_255, ls>0))